In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

from scipy.special import expit

from aml_utils import test_case_checker, perform_computation, show_test_cases

# 0. Data 

Since the MNIST data (http://yann.lecun.com/exdb/mnist/) is stored in a binary format, we would rather have an API handle the loading for us. 

Pytorch (https://pytorch.org/) is an Automatic Differentiation library that we may see and use later in the course. 

Torchvision (https://pytorch.org/docs/stable/torchvision/index.html?highlight=torchvision#module-torchvision) is an extension library for pytorch that can load many of the famous data sets painlessly. 

We already used Torchvision for downloading the MNIST data. It is stored in a numpy array file that we will load easily.

## 0.1 Loading the Data

In [2]:
if os.path.exists('../MeanField-lib/mnist.npz'):
    npzfile = np.load('../MeanField-lib/mnist.npz')
    train_images_raw = npzfile['train_images_raw']
    train_labels = npzfile['train_labels']
    eval_images_raw = npzfile['eval_images_raw']
    eval_labels = npzfile['eval_labels']
else:
    import torchvision
    download_ = not os.path.exists('../MeanField-lib/mnist')
    data_train = torchvision.datasets.MNIST('../MeanField-lib/mnist', train=True, transform=None, target_transform=None, download=download_)
    data_eval = torchvision.datasets.MNIST('../MeanField-lib/mnist', train=False, transform=None, target_transform=None, download=download_)

    train_images_raw = data_train.data.numpy()
    train_labels = data_train.targets.numpy()
    eval_images_raw = data_eval.data.numpy()
    eval_labels = data_eval.targets.numpy()

    np.savez('../MeanField-lib/mnist.npz', train_images_raw=train_images_raw, train_labels=train_labels, 
             eval_images_raw=eval_images_raw, eval_labels=eval_labels) 

In [3]:
noise_flip_prob = 0.04

# <span style="color:blue">Task 2</span>

Write a funciton named `sigmoid_2x` that given a variable $X$ computes the following:

$$f(X) := \frac{\exp(X)}{\exp(X) + \exp(-X)}$$

The input argument is a numpy array $X$, which could have any shape. Your output array must have the same shape as $X$.

**Important Note**: Theoretically, $f$ satisfies the following equations:

$$\lim_{X\rightarrow +\infty} f(X) = 1$$
$$\lim_{X\rightarrow -\infty} f(X) = 0$$

Your implementation must also work correctly even on these extreme edge cases. In other words, you must satisfy the following tests.
* `sigmoid_2x(np.inf)==1` 
* `sigmoid_2x(-np.inf)==0`.

**Hint**: You may find `scipy.special.expit` useful.

divide numerator and denominator of expit $f(X):=\frac{\exp (X)}{\exp (X)+\exp (-X)}$ by $\exp(X)$ to get $\frac{1}{1+\exp(-2X)}$

In [ ]:
def sigmoid_2x(X):
    output = expit(2*X)
    return output

# 1. Applying Mean-field Approximation to Boltzman Machine's Variational Inference Problem

# <span style="color:blue">Task 3</span>

Write a `boltzman_meanfield` function that applies the mean-field approximation to the Boltzman machine. 

Recalling the textbook notation, $X_i$ is the observed value of pixel $i$, and $H_i$ is the true value of pixel $i$ (before applying noise). For instance, if we have a $3 \times 3$ image, the corresponding Boltzman machine looks like this: 

```
       X_1        X_2        X_3
      /          /          /
     H_1 ------ H_2 ------ H_3
      |          |          |
      |          |          |
      |          |          |
      | X_4      | X_5      | X_6
      |/         |/         |/ 
     H_4 ------ H_5 ------ H_6
      |          |          |
      |          |          |
      |          |          |
      | X_7      | X_8      | X_9
      |/         |/         |/ 
     H_7 ------ H_8 ------ H_9
```     

Here, we a adopt a slightly simplified notation from the textbook and define $\mathcal{N}(i)$ to be the neighbors of pixel $i$ (the pixels adjacent to pixel $i$). For instance, in the above figure, we have $\mathcal{N}(1) = \{2,4\}$, $\mathcal{N}(2) = \{1,3,5\}$, and $\mathcal{N}(5) = \{2,4,6,8\}$.


With this, the process in the textbook can be summarized as follows:

```
1. for iteration = 1, 2, 3, ....,
  2. Pick a random pixel i.
  3. Find pixel i's new parameter as
```
$$\pi_i^{\text{new}} = \frac{\exp(\theta_{ii}^{(2)} X_i + \sum_{j\in \mathcal{N}(i)} \theta_{ij}^{(1)} (2\pi_j -1))}{\exp(\theta_{ii}^{(2)} X_i + \sum_{j\in \mathcal{N}(i)} \theta_{ij}^{(1)} (2\pi_j -1)) + \exp(-\theta_{ii}^{(2)} X_i - \sum_{j\in \mathcal{N}(i)} \theta_{ij}^{(1)} (2\pi_j -1))} .$$
```
  4. Replace the existing parameter for pixel i with the new one.
```
$$\pi_i \leftarrow \pi_i^{\text{new}}$$

Since our computational resources are extremely vectorized, we will make the following minor algorithmic modification and ask you to implement the following instead:


1. for iteration = 1, 2, 3, ....,
1. **for each pixels i:**  
1. Find pixel i's new parameter, but do not update the original parameter yet.
  
$$\pi_i^{\text{new}} = \frac{\exp(\theta_{ii}^{(2)} X_i + \sum_{j\in \mathcal{N}(i)} \theta_{ij}^{(1)} (2\pi_j -1))}{\exp(\theta_{ii}^{(2)} X_i + \sum_{j\in \mathcal{N}(i)} \theta_{ij}^{(1)} (2\pi_j -1)) + \exp(-\theta_{ii}^{(2)} X_i - \sum_{j\in \mathcal{N}(i)} \theta_{ij}^{(1)} (2\pi_j -1))} .$$

4. Once you have computed all the new parameters, update all of them at the same time:

$$\pi \leftarrow \pi^{\text{new}}$$

We assume that the parameters $\theta_{ii}^{(2)}$ have the same value for all $i$ and denote their common value by scalar `theta_X`. Moreover, we assume that the parameters $\theta_{ij}^{(1)}$ have the same value for all $i,j$ and denote their common value by scalar `theta_pi`.

The `boltzman_meanfield` function must take the following input arguments:
1. `images`: A numpy array with the shape `(N,height,width)`, where 
    * `N` is the number of samples and could be anything,
    * `height` is each individual image's height in pixels (i.e., number of rows in each image),
    * and `width` is each individual image's width in pixels (i.e., number of columns in each image).
      * Do not assume anything about `images`'s dtype or the number of samples or the `height` or the `width`.
      * The entries of `images` are either -1 or 1.
2. `initial_pi`: A numpy array with the same shape as `images` (i.e. `(N,height,width)`). This variable is corresponding to the initial value of $\pi$ in the textbook analysis and above equations. Note that for each of the $N$ images, we have a different $\pi$ variable.

3. `theta_X`: A scalar with a default value of `0.5*np.log(1/noise_flip_prob-1)`. This variable represents $\theta_{ii}^{(2)}$ in the above update equation.

4. `theta_pi`: A scalar with a default value of 2. This variable represents $\theta_{ij}^{(1)}$ in the above update equation.

5. `iterations`: A scalar with a default value of 100. This variable denotes the number of update iterations to perform.

The `boltzman_meanfield` function must return the final $\pi$ variable as a numpy array called `pi`, and should contain values that are between 0 and 1. 

**Hint**: You may find the `sigmoid_2x` function, that you implemented earlier, useful.



**Hint**: If you want to find the summation of neighboring elements for all of a 2-dimensional matrix, there is an easy and efficient way using matrix operations. You can initialize a zero matrix, and then add four shifted versions (i.e., left-, right-, up-, and down-shifted versions) of the original matrix to it. You will have to be careful in the assignment and selection indices, since you will have to drop one row/column for each shifted version of the matrix. *..and replace that row or column with zeros?*

  * **Important Note**: Do **not** use `np.roll` if you're taking this approach.
  
##### I get lost here:

**Important Note**: When evaluating the neighborhood sum experssions (i.e., terms with $\sum_{j\in \mathcal{N}(i)}$), make sure that you do not inadvertently include a "ghost" pixel in $\mathcal{N}(i)$. For instance, make sure you're only using `H_5`, `H_7`, `H_9`, and `X_8`  when computing an update for `H_8`. That is, only left-, right-, and down-shifted pixels should be contributing to `H_8`'s neighourhood sums. If you mistakenly add an up-shifted pixel to `H_8`'s neighourhood sums (whether it's a copy of `H_8` or a no-ink/zero pixel), you are effectively imposing an extra neighborhood edge between `H_8` and a "ghost" pixel below it; notice that our boltzman machine doesn't have a neighborhood edge between `H_8` and anything below it, therefore, neither `H_8` nor an extra non-inky pixel should be participating in `H_8`'s neighborhood sums and update.
  * Missing this point can cause an initial mismatche in the edge pixel updates, which will be disseminated through iterative updates to other pixels.

##### Notes
1. Create a separate function which uses matrix operations to calculate sum of neighboring elements of pi  
1. For each iteration:  
- For each pi  
  - get neighboring value sum array using step 1 function  
  - Calculate the term for updating parameters:$\theta_{i i}^{(2)} X_{i}+\sum_{j \in \mathcal{N}(i)} \theta_{i j}^{(1)}\left(2 \pi_{j}-1\right)$
3. Pass above term through sigmoid_2x function to get updated value for $\pi$

In [ ]:
def boltzman_meanfield(images, initial_pi, theta_X=0.5*np.log(1/noise_flip_prob-1), theta_pi=2, iterations=100):
    if len(images.shape)==2:
        # In case a 2d image was given as input, we'll add a dummy dimension to be consistent
        X = images.reshape(1,*images.shape)
    else:
        # Otherwise, we'll just work with what's given
        X = images
    
    pi = initial_pi
    
    # Beginning of Mo's code 
    
    #1 Create a separate function which uses matrix operations to calculate sum of neighboring elements of pi
    def shift(pi, right, up):
        e = np.empty_like(pi)
        if right >= 0:
            e[:, :right, :] = 0
            e[:, right:, :] = pi[:, :-right, :]
        else:
            e[:, right:, :] = 0
            e[:, :right, :] = pi[:, -right:, :]
        if up >= 0:
            e[:, :, :up] = 0
            e[:, :, up:] = pi[:, :, :-up]
        else:
            e[:, :, up:] = 0
            e[:, :, :up] = pi[:, :, -up:]
        return e
 
    def SumNi(pi):
        #Ni = shift(pi, 1, 0) + shift(pi, -1, 0) + shift(pi, 0, 1) + shift(pi, 0, -1) #original
        #left[left!=0]=2*left[left!=0]-1 ##from campuswire???
        #do I multiply all pi here by theta_pi?
        r = shift(pi, 1, 0)[shift(pi, 1, 0) !=0]   = 2 * shift(pi, 1, 0)[shift(pi, 1, 0) != 0] - 1 
        l = shift(pi, -1, 0)[shift(pi, -1, 0) !=0] = 2 * shift(pi, -1, 0)[shift(pi, -1, 0) != 0] - 1 
        d = shift(pi, 0, -1)[shift(pi, 0, -1) !=0] = 2 * shift(pi, 0, -1)[shift(pi, 0, -1) != 0] - 1 
        u = shift(pi, 0, 1)[shift(pi, 0, 1) !=0]   = 2 * shift(pi, 0, 1)[shift(pi, 0, 1) != 0] - 1
        Ni = r + l + d + u
        return Ni

    #do I replace pi with theta_pi * (2 * pi - 1)
    for i in range(iterations):
        #2   - get neighboring value sum array using step 1 function  
        #    - Calculate the term for updating parameters
        #cmn_term = theta_X * X + SumNi(pi) * theta_pi * (2 * pi - 1)
        
        cmn_term = theta_X * X + SumNi(theta_pi * (2 * pi - 1))

        #3 Pass above term through sigmoid_2x function to get updated value for pi
        pi = sigmoid_2x(cmn_term)
    
    # End of Mo's code
        
    return pi.reshape(*images.shape)

In [ ]:
# Performing sanity checks on your implementation
def test_boltzman(x, seed = 12345, theta_X=0.5*np.log(1/noise_flip_prob-1), theta_pi=2, iterations=100):        
    np_random = np.random.RandomState(seed=seed)
    initial_pi = np_random.uniform(0,1, size=x.shape)
    return boltzman_meanfield(x, initial_pi, theta_X=theta_X, 
                              theta_pi=theta_pi, iterations=iterations)
    
(orig_image, ref_image, test_im, success_is_row_inky) = show_test_cases(test_boltzman, task_id='3_V')

assert success_is_row_inky

# Checking against the pre-computed test database
test_results = test_case_checker(boltzman_meanfield, task_id=3)
assert test_results['passed'], test_results['message']